# Laboratorium 3: Augmentacje danych i balans klas (DermaMNIST)


**Cele:**
- porównać wpływ różnych **augmentacji** (geometria, intensywność, heavy) na wyniki,
- zastosować metody **balansu klas**: wagi w `CrossEntropy`, `WeightedRandomSampler`,
- przeprowadzić rzetelną **ewaluację** (Loss, Accuracy, Macro-Precision/Recall/F1, Confusion Matrix),
- zebrać wyniki w tabeli i wyciągnąć wnioski.

> **Dataset:** **DermaMNIST (MedMNIST)** – 7-klasowa klasyfikacja zmian skórnych, automatyczny download.

> Uwaga: architektura CNN będzie prosta (dla 28×28×3); skupiamy się na augmentacjach i balansie klas.

Augmentacja danych (ang. data augmentation) to zestaw technik sztucznego powiększania i urozmaicania zbioru treningowego poprzez modyfikacje istniejących danych. Istniejące próbki przekształca się tak, aby powstały ich różnorodne warianty, ale zachowujące tę samą etykietę (klasę). Przykłady dla obrazów:
- obrót, odbicie lustrzane, przycięcie, skalowanie, przesunięcie,
- zmiana jasności, kontrastu, koloru, dodanie szumu,
- losowe maskowanie fragmentów obrazu (Cutout, Random Erasing).

Stosowana jest ona w celu: zwiększenia rozmiaru zbioru treningowego – szczególnie ważne, gdy mamy mało przykładów, poprawy generalizacji – model uczy się odporniejszych cech (mniejszy overfitting), symulacji warunków rzeczywistych – różne oświetlenia, kąty patrzenia czy szumy w danych, wyrównania zbioru – generowanie dodatkowych przykładów dla klas, których jest mniej.

## 0) Instalacja i importy

In [ ]:
# !pip -q install medmnist torchmetrics tqdm scikit-learn pandas

import os, random, math, json, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms
from tqdm.auto import tqdm

from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             confusion_matrix, ConfusionMatrixDisplay)

import medmnist
from medmnist import DermaMNIST, INFO

SEED = 42
def set_seed(seed=SEED):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
set_seed()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', device)

## 1) Pobranie danych (DermaMNIST, MedMNIST) + szybki podgląd i rozkład klas

Skrypt automatycznie pobiera zbiór **DermaMNIST**. Poniższy kod sprawdza rozmiar danych i etykiety.
Wyświetlone zostaje również 6 pierwszych próbek z tego zbioru.

Zwróć uwagę na rozkład klas w wykorzystywanym zbiorze. Co można o nim powiedzieć?

In [ ]:
DATA_ROOT = './data/dermamnist'
os.makedirs(DATA_ROOT, exist_ok=True)
info = INFO['dermamnist']
NUM_CLASSES = len(info['label'])
print('Opis:', info['description'])
print('Liczba klas:', NUM_CLASSES, '; label map:', info['label'])

# as_rgb=True -> 3 kanały
train_raw = DermaMNIST(split='train', download=True, root=DATA_ROOT, as_rgb=True)
val_raw   = DermaMNIST(split='val',   download=True, root=DATA_ROOT, as_rgb=True)
test_raw  = DermaMNIST(split='test',  download=True, root=DATA_ROOT, as_rgb=True)
print('Rozmiary:', len(train_raw), len(val_raw), len(test_raw))

# podgląd kilku przykładów
fig, axes = plt.subplots(1, 6, figsize=(10,2))
for i in range(6):
    img, y = train_raw[i]
    axes[i].imshow(np.array(img))
    axes[i].set_title(f'y={int(y.squeeze().item())}')
    axes[i].axis('off')
plt.tight_layout(); plt.show()

# Rozkład klas w train
vals, cnts = np.unique(train_raw.labels.squeeze(), return_counts=True)
print('Rozkład klas (train):', dict(zip(vals.tolist(), cnts.tolist())))

## Zadanie 1 – Dataset z przełączanymi augmentacjami i normalizacją

W następnym fragmencie kodu chcemy przygotować różn warianty augmentacji danych, które będą wykorzystane do uczenia sieci i porównania wyników. W tym celu korzystamy z klasy `transforms` z biblioteki `torchvision`.

1. Przygotuj najprostszą, bazową transformację danych. Zawiera ona przekształcenia, które będą obecne we wszystkich innych wariantach augmentacji.
2. Do stworzenia pojedynczego wariantu wykorzystujemy funkcję `transforms.Compose`. Przyjmuje ona listę transformacji, które powinny być wykonane na wczytywanych danych.
3. Podstawowy wariant powinien zawierać dwie transformacje: `transforms.ToTensor()` i `transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])`. Pierwsza z nich przekształca dane do typu tensora z PyTorch z odpowiednią kolejnością wymiarów. Dodatkowo dla obrazów typu `uint8` zmienia ich zakres do `[0.0, 1.0]`. Jeśli wejściem są wartości typu `float`, to zostaną one zachowane. Druga z kolei zmienia zakres danych do przedziału `[-1.0, 1.0]` (zakłądając, że wejście ma zakres `[0.0, 1.0]`). Jej arguemntami są paramtry normalizacji dla poszczególnych kanałów.
4. Stwórz transformację, która przed bazowymi wykona dodatkowo odbicie w poziomie `transforms.RandomHorizontalFlip` i rotację obrazu `transforms.RandomRotation`. Argumentem pierwszej jest prawdopodobieństwo odbicia, a drugiej maksymalny kąt rotacji w stopniach.
5. Stwórz transformację, która przed bazowymi wykona dodatkowo losową zmianę kolorów `transforms.ColorJitter`. Argumentami są maksymalne zmiany dla danej właściwości, np. `transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.05, hue=0.02)`.
6. Czwarta transformacja powinna łączyć wszystkie wymienione wcześniej.
7. Napisz również funkcję `make_datasets`, która będzie tworzyć DataSet i aplikować transformacje przekazane jako argument do danych treningowych. Zwracać powinna 3 zbiory danych: treningowy, walidacyjny i testowy. Dla danych walidacyjnych i testowych stosujemy tylko podstawowy wariant transformacji. Transformacja jest przekazywana jako dodatkowy arguement podczas inicjalizacji obiektu klasy `DermaMNIST`.
8. Zdefiniuj `BATCH_SIZE`, `NUM_WORKERS` i `PIN_MEMORY`.

In [ ]:
# Zadanie 1

## Zadanie 2 – Prosta sieć CNN 3-kanałowa + pętle trening/ewaluacja

Zdefniujmy bardzo podobną sieć do wykorzystanej w poprzednim tygodniu. Poniżej znajduje się lista modyfikacji koniecznych do wykonania.

1. Liczba kanałów wejściowych: 3.
2. Liczba kanałów wyjściowych pierwszej warstwy konwolucyjnej: 16.
3. Liczba kanałów wyjściowych drugiej warstwy konwolucyjnej: 32.
4. Rozmiar obu konwolucji wynosi 3, a padding 1.
5. Wyjście pierwszej warstwy w pełni połączonej ma rozmiar 128.
6. Liczba klas (a więc i rozmiar wyjścia sieci) powinna być podawana jako argument.
7. Dodajmy również warstwę dropout o prawdopodobieństwie równym 0.25 `nn.Dropout(p=0.25)` pomiędzy warstwami w pełni połączonymi. Warstwa ta losowo wyłącza część neuronów z zadanym prawdopodobieństwem. Warstwa taka może poprawić generalizację sieci i sprawić, że będzie się uczyć bardziej ogólnych cech.
8. Zaimplementuj funkcję, która będzie wykonywać jedną epokę treningu (podobnie jak w poprzednim ćwiczeniu). Jej argumentami są trenowana sieć, DataLoader, optimalizator i obiekt funkcji straty. Nie musi ona liczyć dokładności, bo zaimplementujemy osobną funkcję do ewaluacji.
9. Zaimplementuj funkcję wykonującą ewaluację sieci. Jak argumenty przyjmuje ona instancję sieci i DataLoader. Zapamiętaj w dwóch listach predykcje sieci i etykiety. Po przetworzeniu danych połącz je w jeden wektor za pomocą funkcji `np.concatenate` i oblicz metryki klasyfikacji takie jak: dokładność (accuracy), precyzja (precision), czułość (recall) i F1-score. W tym calu wykozystaj funkcje `accuracy_score`, `precision_score`, `recall_score` i `f1_score`.
    - Dokładność jest odsetkiem poprawnie sklasyfikowanych próbek.
    - Precyzja mówi ile z przewidzianych klasyfikacji danej klasy naprawdę należało do danej klasy. Dodajemy argument `average='macro`, który sprawia, że finalny wynik jest uśrednioną wartością dla wszystkich klas.
    - Czułość mówi ile z prawdziwych przykładów danej klasy zostało poprawnie znalezionych przez model. W tym przypadku również chcemy uśrednić wyniki dla wszystkich klas.
    - F1-score jest średnią harmoniczną z precyzji i czułości. Również chcemy go uśrednić dla wszystkich klas.
Na wyjście sieci przekaż słownik zawierający obliczone metryki, wektor predykcji sieci oraz wektor rzeczywistych etykiet.

Dlaczego w przypadku wykorzystywanego zbioru po prostu sprawdzanie dokładności może nie być dobrą metodą oceny jej skuteczności?

In [ ]:
# Zadanie 2

## Zadanie 3 – Przeprowadzenie treningu i sprawdzenie augmentacji

Przeprowadź trening zdefiniowanej sieci dla każdego z wariantów augmentacji danych treningowych. Dla wielu przypadków testowych dobrze jest stworzyć strukturę zawierającą scenariusze testowe i uruchomić je w pętli.

1. Dla każdego pzypadku stwórz 3 DataSety z odpowiednimi transformacjami (użyj przygotowanej wcześniej funkcji), a następnie dla każdego z nich DataLoader.
2. Dla każdego przypadku stwórz instancję trenowanej sieci, optymalizator `torch.optim.Adam` i funkcję straty `nn.CrossEntropyLoss`.
3. Stwórz puste listy do śledzenia postępu treningu: `train_losses`, `val_losses`, `train_accuracies`, `val_accuracies`, `val_f1_scores`, `epochs_list`.
4. Dodatkowo chcemy zapamiętać model, który dla danego treningu osiągnął najlepszą wartość metryki F1-macro. Wymaga to zapamiętania najlepszej do tej pory widzianej metryki i poprawnej inicjalizacji.
5. Napisz pętlę trenującą. W każdej iteracji chcemy:
    - wykonać epokę treningu,
    - obliczyć i zapamiętać metryki dla zbioru treningowego,
    - obliczyć i zapamiętać metryki dla zbioru walidacyjnego,
    - obliczyć i zapamiętać stratę dla zbioru treningowego,
    - obliczyć i zapamiętać stratę dla zbioru walidacyjnego,
    - sprawdzić czy jest to najlepszy do tej pory model,
    - (opcjonalnie) wyświetlać wyniki po każdej epoce.
6. Po zakończonym treningu wyświetl 2 wykresy:
    - wykres staty dla zbioru treningowego i walidacyjnego,
    - wykres wyznaczonych metryk dla zbioru treningowego i walidayjnego.
7. Wykonaj ewaluację dla zbioru testowego. Wyświetl macierz pomyłek (podobnie jak w poprzednim ćwiczeniu).

Dobrze jest zapamiętywać wyniki metryki zbioru testowego dla wszystkich testowanych przypadków i wyświetlić je na końcu w formie tabeli.

In [ ]:
# Zadanie 3

## Zadanie 4 – Balans klas: wagi, WeightedRandomSampler

Wykorzystamy dodatkowo dwie metody radzenia sobie ze zbiorami danych i niezbalansowanych klasach.
Pierwszą są wagi klas (class weights). Działa to w ten sposób, że jeśli jakaś klasa jest rzadka, to jej błędy mają być bardziej kosztowne. Dodając większe wagi dla rzadkich klas w funkcji straty, uczymy model, żeby przykładał do nich większą wagę.

Drugą metodą jest zmiana doboru próbek (WeightedRandomSampler). W tym przypadku rzadkie klasy są częściej losowane do batcha, a więc model widzi ich proporcjonalnie więcej. Również w tym przypadku tworzymy wagi dla każdej klasy.

1. Zaimplementuj funkcję, która będzie obliczać wagi dla klas. Jej argumentem jest lista rzeczywistych etykiet zbioru treningowego (metoda `.labels` dla zbioru treningowego).
2. Wewnątrz funkcji sprawdzamy ile jest wystąpień dla każdej etykiety (funkcja `np.unique`). Jej pierwszym argumentem są etykiety. Przekazujemy jej również argument `return_counts=True`, który powoduje, że funkcja zwraca również liczbę wystąpień.
3. Następnie dzielimy otrzymane wystąpienia przez liczbę próbek. W ten sposób obliczamy częstość występowania danej klasy.
4. Chcemy, aby waga była odwrotnie proporcjonalna do częstości jej występowania, a więc obliczamy odwrotność poprzedniej wartości. Warto dodać zabezpieczenie przed zbyt małą wartością mianownika `np.maximum(freq, 1e-8)`.
5. Normalizujemy wartości obliczonych wag tak, żeby ich suma wynosiła liczbę możliwych etykiet.
6. Zwracamy tensor wag `torch.tensor` z danymi typu `torch.float32`.
7. Wyświetl obliczone wagi.
8. Na podstawie wag zwróconych przez zaimplementowaną funkcję stworzymy sampler do danych treningowych. W tym celu należy obliczyć wagę dla każdej próbki ze zbioru treningowego. Bierzemy więc listę etykiet próbek i sprawdzamy jakie prawdopodobieństwo odpowiada każdej z nich (a więc robimy Look-Up Table).
9. Obliczone nagi dla każdej próbki przezkazujemy do klasy `WeightedRandomSampler` jako argument `weights`. Oprócz tego konieczne jst przekazanie argumentu `num_samples`, czyli liczba próbek, i podanie argumentu `replacement` jako `True`.

In [ ]:
# Zadanie 4

## Zadanie 5 – Przeprowadzenie treningu i sprawdzenie wpływu testowanych metod na wynik

Przeprowadzamy trening podobno do zadania 3, ale tym razem zamiast metod augumentacji sprawdzamy wpływ wag strat i częstości doboru próbek.

1. Aby sprawdzić wpływ wag, do funkcji straty należy przekazać ich wektor `nn.CrossEntropyLoss(weight=weights_ce.to(device))`.
2. Aby sprawdzić wpływ samplera, należy go przekazać do DataLoadera jako argument `sampler`.
3. Sprawdź wpływ zaimplementowanych metod na wyniki treningu przeprowadzając eksperymenty w podobny sposób jak w zadaniu 3.

In [ ]:
# Zadanie 5

## Zadanie 6 – Eksperymenty własne

Spróbuj połączyć poznane metody tak, aby uzysać jak największy wskaźnik F1-macro. Możesz dodatkowo użyć innych metod augmentacji danych, takich jak: `transforms.RandomVerticalFlip`, `transforms.RandomAffine`, `transforms.RandomResizedCrop`, `transforms.GaussianBlur`, `transforms.RandomApply`, `transforms.RandomAdjustSharpness`, `transforms.RandomGrayscale`, `transforms.RandomErasing`, `transforms.Lambda` (może zostać użyte do uzyskania szumu Gaussowskiego: `transforms.Lambda(lambda x: torch.clamp(x + 0.03*torch.randn_like(x), -1, 1))`, w tej formie używana już po normalizacji zakresu), `transforms.TrivialAugmentWide`, `transforms.RandAugment`.

Pochwal się uzyskanym wynikiem.

In [ ]:
# Zadanie 6

## Podsumowanie – Wnioski

- Która strategia **augmentacji** dała najwyższą Macro-F1?
- Czy **wagi** w CE lub **sampler** poprawiły Recall klas rzadkich?
- Jakie klasy najczęściej się mylą (macierz pomyłek)?